In [2]:
import os
import sqlite3
from multiprocessing import freeze_support
from modin.db_conn import ModinDatabaseConnection
import modin.pandas as mpd
%load_ext autoreload
%autoreload 2
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
def load_data_from_db(con):
    try:
        df = mpd.read_sql("SELECT * FROM data", con)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


freeze_support()
dbfile = '/workspace/code/train_data_for_competition/mini_training_set.db'

conn = ModinDatabaseConnection('sqlalchemy', f'sqlite:///{dbfile}')

# Can use get_connection to get underlying sqlalchemy engine
conn.get_connection()
db_df = load_data_from_db(conn)
print(db_df.head())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-03-09 00:33:53,439	INFO worker.py:1841 -- Started a local Ray instance.


   Device_ID                   Datetime      URL  Domain_Name  Domain_cls1  \
0        124  2023-04-23 03:04:30+03:00     6466      2368671          755   
1        124  2023-04-23 03:04:30+03:00  2245864      1792903            0   
2        124  2023-04-23 03:04:30+03:00  1839478       107342          332   
3        124  2023-04-23 03:14:50+03:00  1172090       107342          332   
4        124  2023-04-23 03:14:50+03:00  1839478       107342          332   

   Domain_cls2  Domain_cls3  Domain_cls4  Target  
0          799            0            0       0  
1            0            0            0       0  
2            0            0            0       0  
3            0            0            0       0  
4            0            0            0       0  


(raylet) Spilled 5659 MiB, 183 objects, write throughput 310 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


In [3]:
# "23-04 to 18-05"
db_df["Datetime"] = mpd.to_datetime(db_df["Datetime"])
db_df["Datetime"]
# db_df.groupby("Device_ID").apply(lambda x: (x-x["Datetime"].min()).dt.days)

0          2023-04-23 03:04:30+03:00
1          2023-04-23 03:04:30+03:00
2          2023-04-23 03:04:30+03:00
3          2023-04-23 03:14:50+03:00
4          2023-04-23 03:14:50+03:00
                      ...           
32248978   2023-05-13 21:26:02+03:00
32248979   2023-05-13 21:26:03+03:00
32248980   2023-05-13 21:32:33+03:00
32248981   2023-05-13 21:32:38+03:00
32248982   2023-05-13 21:32:43+03:00
Name: Datetime, Length: 32248983, dtype: datetime64[ns, UTC+03:00]

In [8]:
# %%writefile preprocessing.py
def get_train_test_masks(domain_counts, test_size=0.2, random_state=42):
    # Get unique device IDs and their corresponding targets
    device_target_df = domain_counts.groupby('Device_ID')['Target'].first().reset_index()
    
    # Perform stratified split on device IDs
    train_device_ids, test_device_ids = train_test_split(
        device_target_df['Device_ID'],
        test_size=test_size,
        random_state=random_state,
        stratify=device_target_df['Target']
    )
    
    # Create mask for train/test split in domain_counts
    train_mask = domain_counts['Device_ID'].isin(train_device_ids)
    test_mask = domain_counts['Device_ID'].isin(test_device_ids)
    
    # Print statistics
    print(f"Total devices: {len(device_target_df)}")
    print(f"Train devices: {len(train_device_ids)}")
    print(f"Test devices: {len(test_device_ids)}")
    print(f"\nTrain samples: {len(domain_counts[train_mask])}")
    print(f"Test samples: {len(domain_counts[test_mask])}")
    
    # Print class distribution
    print("\nTarget distribution in train set:")
    print(domain_counts[train_mask].groupby('Target').size() / len(domain_counts[train_mask]))
    print("\nTarget distribution in test set:")
    print(domain_counts[test_mask].groupby('Target').size() / len(domain_counts[test_mask]))
    
    return train_mask, test_mask

def get_domain_counts(db_df, pivot=False):
    domain_counts = db_df.groupby(["Device_ID","Domain_Name","Target"]).count()
    domain_counts = domain_counts.reset_index()
    domain_counts = domain_counts[["Device_ID","Domain_Name","Target","Datetime"]]
    domain_counts.rename(columns={"Datetime":"count"}, inplace=True)
    if pivot:
        pivot_matrix = train_domain_counts.pivot(index='source', columns='target', values='count').fillna(0)
        return pivot_matrix
    return domain_counts
def get_device_domain_fractions(domain_counts):
    # Calculate total counts per device
    device_totals = domain_counts.groupby('Device_ID')['count'].sum()
    
    # Calculate fractions by dividing each count by the device total
    domain_fractions = domain_counts.copy()
    domain_fractions['fraction'] = domain_fractions.apply(
        lambda row: row['count'] / device_totals[row['Device_ID']], 
        axis=1
    )
    
    return domain_fractions[['Device_ID', 'Domain_Name', 'Target', 'count', 'fraction']]
def compute_domain_target_correlation(domain_counts):
    # Group by Domain_Name and calculate mean Target and count
    domain_stats = domain_counts.groupby('Domain_Name').agg({
        'Target': 'mean',
        'count': ['mean', 'std', 'count']
    }).reset_index()
    
    # Flatten column names
    domain_stats.columns = ['Domain_Name', 'target_mean', 'count_mean', 'count_std', 'n_devices']
    
    # Calculate correlation coefficient
    # We only include domains that appear in multiple devices for statistical significance
    significant_domains = domain_stats[domain_stats['n_devices'] > 1]
    
    # Calculate correlation and p-value
    correlation = mpd.DataFrame({
        'Domain_Name': significant_domains['Domain_Name'],
        'target_correlation': significant_domains['target_mean'],
        'avg_count': significant_domains['count_mean'],
        'count_std': significant_domains['count_std'],
        'n_devices': significant_domains['n_devices']
    }).sort_values('target_correlation', ascending=False)
    
    return correlation

In [ ]:
print("keep in mind that the best resolution will be achieved with a resolution of urls/chain of urls, not domains")
print("Cluster url walks")
print("I want to cluster urls/url walks from a given domain, to 3 categories")
print("positive correlation, zero correlation, negative correlation, to_label")

### Getting Basic features ( Domain Counts )

In [9]:
domain_counts = get_domain_counts(db_df)
del db_df
pivot_matrix = domain_counts.pivot(index='Domain_Name', columns='Device_ID', values='count').fillna(0)


In [37]:
print("user use fraction instead of count")

user use fraction instead of count


In [ ]:
domain_counts

,Device_ID,Domain_Name,Target,count
0,124,3930,0,4
1,124,4136,0,3
2,124,6450,0,3
3,124,12837,0,1
4,124,17665,0,2
...,...,...,...,...
288003,69967,2361028,1,4
288004,69967,2387835,1,34
288005,69967,2389761,1,72
288006,69967,2390487,1,1


In [22]:
# %%writefile correlation.py
import numpy as np
import ray
@ray.remote
def calculate_chunk_covariance(matrix_centered, start_idx, end_idx, columns):
    # Calculate covariance for this chunk
    chunk = matrix_centered[:, start_idx:end_idx]
    chunk_cov = np.dot(chunk.T, matrix_centered) / (matrix_centered.shape[0] - 1)
    
    return start_idx, end_idx, chunk_cov
def compute_chunked_covariance(pivot_matrix, batch_size=2000):
    # Convert to numpy array for faster computation
    matrix_dense = pivot_matrix.to_numpy()
    matrix_centered = matrix_dense - np.mean(matrix_dense, axis=0)

    # Initialize parameters
    n_cols = pivot_matrix.shape[1]
    futures = []

    # Submit tasks to Ray
    for i in range(0, n_cols, batch_size):
        batch_end = min(i + batch_size, n_cols)
        futures.append(calculate_chunk_covariance.remote(matrix_centered, i, batch_end, pivot_matrix.columns))

    # Collect results and combine
    cov_chunks = []
    for future in ray.get(futures):
        start_idx, end_idx, chunk_cov = future
        chunk_df = mpd.DataFrame(
            chunk_cov,
            index=pivot_matrix.columns[start_idx:end_idx],
            columns=pivot_matrix.columns
        )
        cov_chunks.append(chunk_df)

    # Combine all chunks
    return mpd.concat(cov_chunks)
def melt_covariance_matrix(covariance_matrix):
    # Reset index to make it a column
    melted = covariance_matrix.reset_index()
    
    # Melt the dataframe
    melted = melted.melt(
        id_vars=['Device_ID'],
        var_name='target',
        value_name='covariance'
    )
    
    # Rename the 'index' column to 'source'
    melted = melted.rename(columns={'Device_ID': 'source'})
    
    # Remove duplicate pairs (e.g., if A->B exists, remove B->A)
    melted = melted[melted['source'] < melted['target']]
    
    # Remove rows where source equals target
    melted = melted[melted['source'] != melted['target']]
    
    return melted.reset_index(drop=True)
def filter_and_transform_covariance(melted_covariance, abs_threshold=0.03, log_threshold=10):
    # Create a copy to avoid modifying the original dataframe
    result = melted_covariance
    # Create corrected_cov column
    result['corrected_cov'] = result['covariance'].copy()
    
    # Apply absolute threshold filter
    result.loc[abs(result['corrected_cov']) < abs_threshold, 'corrected_cov'] = 0
    
    # Apply log transformation for values above log_threshold
    high_vals_mask = abs(result['corrected_cov']) > log_threshold
    result.loc[high_vals_mask, 'corrected_cov'] = result.loc[high_vals_mask, 'corrected_cov'].apply(
        lambda x: np.log(abs(x)) * np.sign(x)
    )
    print(high_vals_mask.mean())
    
    return result

### Calculating Covariance/Correlation between domains


In [23]:
# Get domain counts and create pivot matrix

# Compute covariance matrix and melt it
covariance_matrix = compute_chunked_covariance(pivot_matrix)
melted_cov = melt_covariance_matrix(covariance_matrix)

# Apply filtering and transformation to covariance
corrected_cov = filter_and_transform_covariance(melted_cov)

# Get device domain fractions from original domain counts
device_fractions = get_device_domain_fractions(domain_counts)
# Compute domain-target correlations
domain_correlations = compute_domain_target_correlation(domain_counts)

print("Shapes:")
print(f"Corrected covariance: {corrected_cov.shape}")
print(f"Device fractions: {device_fractions.shape}")

0.996048833452504
Shapes:
Corrected covariance: (188805, 4)
Device fractions: (288008, 5)


In [ ]:
%%writefile graph_model_funcs.py
from sklearn.model_selection import train_test_split
import torch
from torch_geometric.nn import Node2Vec
from torch_geometric.data import Data
import networkx as nx



def create_graph_data(corrected_cov):
    # Convert domain names to numerical indices
    unique_domains = pd.concat([corrected_cov['source'], corrected_cov['target']]).unique()
    domain_to_idx = {domain: idx for idx, domain in enumerate(unique_domains)}
    
    # Create edge index and edge weights
    edge_index = torch.tensor([
        [domain_to_idx[s] for s in corrected_cov['source']],
        [domain_to_idx[t] for t in corrected_cov['target']]
    ], dtype=torch.long)
    
    edge_weight = torch.tensor(corrected_cov['corrected_cov'].values, dtype=torch.float)
    
    # Create PyTorch Geometric Data object
    data = Data(
        edge_index=edge_index,
        edge_attr=edge_weight,
        num_nodes=len(unique_domains)
    )
    return data, domain_to_idx

def train_node2vec(data, device='cuda', epochs=100):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = Node2Vec(
        data.edge_index,
        embedding_dim=128,
        walk_length=20,
        context_size=10,
        walks_per_node=10,
        p=1,
        q=1,
        sparse=True
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = model.loss()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')
            
    return model
@ray.remote
def _compute_device_embedding(device_id, group, embeddings, domain_mapping):
    valid_embeddings = []
    valid_weights = []
    
    for _, row in group.iterrows():
        domain = row['Domain_Name']
        if domain in domain_mapping:
            idx = domain_mapping[domain]
            valid_embeddings.append(embeddings[idx])
            valid_weights.append(row['fraction'])
    
    if valid_embeddings:
        valid_weights = np.array(valid_weights)
        valid_weights = valid_weights / valid_weights.sum()
        device_embedding = np.average(valid_embeddings, weights=valid_weights, axis=0)
        return device_id, device_embedding
    return device_id, None

def compute_device_embeddings(device_fractions, embeddings, domain_mapping):
    # Group device fractions by Device_ID
    grouped_fractions = device_fractions.groupby('Device_ID')
    
    # Create remote tasks
    futures = [
        _compute_device_embedding.remote(device_id, group, embeddings, domain_mapping)
        for device_id, group in grouped_fractions
    ]
    
    # Collect results
    results = ray.get(futures)
    
    # Convert results to dictionary
    device_embeddings = {
        device_id: embedding 
        for device_id, embedding in results 
        if embedding is not None
    }
    
    return device_embeddings

# Main flow


### Train Node2Vec

In [ ]:
train_mask, test_mask = get_train_test_masks(domain_counts)
data, domain_mapping = create_graph_data(corrected_cov)

model = train_node2vec(data)
embeddings = model().detach().cpu().numpy()

### Infer Test Embeddings